In [1]:
using LinearAlgebra, Plots, Printf, SparseArrays, IterativeSolvers, LaTeXStrings

The conjugate gradient algorithm

In [50]:
function CG(f,b,⋄,eps::Float64;kmax = 1e6, resflag = false)
   x = 0.0*b; r = b; p = r; n = 0;
   for j = 1:kmax
        n = j
        q = f(p)
        a = (r⋄r)/(p⋄q)
        x = x + a*p
        r_old = r
        r = r - a*q
        if resflag
            println(sqrt(r⋄r))
        end
        if sqrt(r⋄r) < eps
            break
        end
        b = (r⋄r)/(r_old⋄r_old)
        p = r + b*p
        
    end
    @printf("CG took %i iterations",n)
    x
end

CG (generic function with 3 methods)

In [3]:
A = SymTridiagonal(fill(2.0,10),fill(-1.0,9));
b = randn(10);

In [4]:
x = CG(A,b,1e-16);
x - A\b |> norm

CG took 11 iterations

1.3710242980056706e-15

The conjugate gradient algorithm for general linear functions and inner products

In [51]:
function CG(f,g,b,⋄,eps::Float64;kmax = 1e6, resflag = false)
   x = 0.0*b; r = b; n = 0; z = g(r);  p = z;
   for j = 1:kmax
        n = j
        w = f(p)
        a = (z⋄r)/(p⋄w)
        x = x + a*p
        r_old = r
        z_old = z
        r = r - a*w
        if resflag
            println(sqrt(r⋄r))
        end
        if sqrt(r⋄r) < eps
            break
        end
        z = g(r)
        b = (z⋄r)/(z_old⋄r_old)
        p = z + b*p 
    end
    @printf("CG took %i iterations \n",n)
    x
end

CG (generic function with 3 methods)

In [6]:
f = x -> A*x
function ⋄(x,y)
    x'*y
end

⋄ (generic function with 1 method)

In [7]:
x = CG(f,b,⋄,1e-16)
A\b - x |> norm

CG took 11 iterations

1.3710242980056706e-15

Sylvester matrix equations

$$ A X + X B = C.$$

In [8]:
n = 400;
C = randn(n,n);
A = SymTridiagonal(fill(2.0,n),fill(-1.0,n-1));

In [9]:
f = x -> A*x + x*A
function ⋄(x,y)
    dot(x,y)
end

⋄ (generic function with 1 method)

In [10]:
400^2

160000

In [11]:
X = CG(f,C,⋄,1e-16)

CG took 1932 iterations

400×400 Matrix{Float64}:
 -0.292761   -0.301778   -0.0585905  …  -0.255535  -0.181207  -0.0623019
  0.0674041  -0.341596   -0.0382994     -0.275753  -0.338181  -0.276095
  0.277934    0.0933124   0.266232      -0.319016  -0.385177   0.00677955
  0.667992    0.360568    0.587438      -0.670215  -0.285719   0.0091249
  0.622536    0.836138    0.939917      -0.89596   -0.636492  -0.50535
  1.34955     1.51318     1.21467    …  -1.67871   -0.774428  -0.53889
  1.19384     1.35076     0.933788      -1.95738   -1.34777   -1.04857
  0.824516    1.02891     0.495674      -2.37541   -1.91975   -0.863564
  0.111268    0.347568   -0.049965      -2.63596   -1.87745   -0.779982
 -0.326915   -0.164533   -0.20895       -3.01238   -2.30556   -1.57703
  0.436721   -0.426621   -0.0438447  …  -2.93981   -2.45286   -1.36229
  0.636102    0.397783    0.130832      -3.43234   -2.47323   -1.27447
  0.690414    0.270047    0.20824       -3.76496   -2.7621    -1.20879
  ⋮                                  ⋱    

Preconditioned CG

In [12]:
function CG(f,g,b,⋄,eps::Float64)
   x = 0.0*b; r = b; n = 0; z = g(r);  p = z;
   for j = 1:1e6
        n = j
        w = f(p)
        a = (z⋄r)/(p⋄w)
        x = x + a*p
        r_old = r
        z_old = z
        r = r - a*w
        if sqrt(r⋄r) < eps
            break
        end
        z = g(r)
        b = (z⋄r)/(z_old⋄r_old)
        p = z + b*p 
    end
    @printf("CG took %i iterations \n",n)
    x
end

CG (generic function with 3 methods)

Test in trivial case

In [13]:
A = SymTridiagonal(fill(2.0,10),fill(-1.0,9));
b = randn(10);

In [14]:
f = x -> A*x
function ⋄(x,y)
    dot(x,y)
end
g = x -> A\x

#7 (generic function with 1 method)

In [15]:
x = CG(f,g,b,⋄,1e-15)

CG took 1 iterations 


10-element Vector{Float64}:
 -1.0481084529847624
 -0.7172522557586231
  0.11550792140461674
  0.0719361443739376
  0.016515435735016345
  0.7285848279710132
  1.0908412213629224
  0.7734308438652752
 -0.5336302979831764
 -0.5637861903880412

Back to the Sylvester equation

In [45]:
m = 1999;
h = 1/(m+1)

0.0005

In [46]:
C = spzeros(m,m);
C[1,:] += rand(m)
C[end,:] += rand(m)
C[:,1] += rand(m)
C[:,end] += rand(m)
C *= h^(-2)
A = SymTridiagonal(fill(2.0,m),fill(-1.0,m-1))/h^2;

In [47]:
f = X -> A*X + X*A
function ⋄(X,Y)
    h^2*dot(X,Y)
end
g = X -> (.5A+I/h^2)\((.5A+I/h^2)\X')'
#g = X -> (A+2I/h^2)\X
g = X -> h^2/4*X

#41 (generic function with 1 method)

In [53]:
@time X1 = CG(f,C,⋄,h^2; kmax = 100, resflag = false);

CG took 100 iterations  8.151900 seconds (3.75 k allocations: 26.796 GiB, 23.22% gc time)


In [40]:
49*49

2401

In [54]:
@time X2 = CG(f,C,⋄,1e-4; resflag = true, kmax = 10);

61627.80233470201
44333.23907392437
33796.10889754335
27970.53421134213
23307.057691225596
20410.133256160436
17792.93153728655
16060.12573254482
14388.535247511749
13237.050914722447
CG took 10 iterations  1.212614 seconds (6.97 k allocations: 2.681 GiB, 29.28% gc time, 0.89% compilation time)


In [55]:
g = X -> h^2/4*X
@time X2 = CG(f,g,C,⋄,1e-4; resflag = true, kmax = 10);

61627.80233470231
44333.23907392458
33796.108897543505
27970.534211342256
23307.057691225706
20410.133256160516
17792.931537286615
16060.12573254488
14388.535247511803
13237.050914722495
CG took 10 iterations 
  1.148125 seconds (158.42 k allocations: 2.987 GiB, 18.37% gc time, 7.62% compilation time)


Jacobi

In [24]:
Ad = SymTridiagonal(fill(0.0,m),fill(-1.0,m-1))*1/4;
Cd = C*h^2/4;

In [25]:
f_jac = X -> -Ad*X - X*Ad
X = 0*C;

In [26]:
for i = 1:150000
    Xnew = f_jac(X) + Cd
    if norm(X-Xnew) < 1e-5
        @printf("Jacobi took %i iterations",i)
        break
    end
    X = Xnew
end

Jacobi took 4206 iterations